In [1]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 4.3 MB/s eta 0:00:00


In [2]:
!pip install webdriver-manager

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--no-sandbox')  # Required for Colab
options.add_argument('--disable-dev-shm-usage')  # Overcome limited resources
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y
!which google-chrome

--2025-01-09 05:16:10--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.250.141.136, 142.250.141.190, 142.250.141.91, ...
Connecting to dl.google.com (dl.google.com)|142.250.141.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112846176 (108M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb.1’

google-chrome-stabl 100%[===================>] 107.62M   215MB/s    in 0.5s    

2025-01-09 05:16:11 (215 MB/s) - ‘google-chrome-stable_current_amd64.deb.1’ saved [112846176/112846176]

(Reading database ... 123787 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (131.0.6778.264-1) over (131.0.6778.264-1) ...
Setting up google-chrome-stable (131.0.6778.264-1) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Rea

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# Set up Chrome in headless mode
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = "/usr/bin/google-chrome"  # Explicitly set the Chrome binary location

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def scrape_medicines():
    # Scrape the list of medicine details
    medicines = driver.find_elements(By.CSS_SELECTOR, 'a.show-loader')
    for medicine in medicines:
        try:
            # Attempt to extract details
            name = medicine.find_element(By.CSS_SELECTOR, 'span.name').text
            price = medicine.find_element(By.CSS_SELECTOR, 'span.price').text
            image_url = medicine.find_element(By.TAG_NAME, 'img').get_attribute('src')

            # Print or save the data
            print(f"Name: {name}, Price: {price}, Image URL: {image_url}")
        except NoSuchElementException as e:
            print(f"Skipping medicine due to missing element: {e}")

def scrape_a_to_z():
    # Scrape data for each letter A to Z
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        url = f"https://healthwire.pk/pharmacy/medicines/{letter}"
        print(f"Scraping data for letter: {letter} ({url})")

        driver.get(url)
        sleep(2)  # Wait for the page to load

        # Scrape the medicine data
        scrape_medicines()

try:
    scrape_a_to_z()
finally:
    driver.quit()


Scraping data for letter: a (https://healthwire.pk/pharmacy/medicines/a)
Name: Alp (0.5mg) 30 Tablets, Price: Rs. 116.1, Image URL: https://d3313lwq5y3sh2.cloudfront.net/assets/photos/001/084/282/pharmacy_online_small/ALP_0.5MG_TAB__1.webp?1673597790
Name: Ascard (75mg) 30 Tablets, Price: Rs. 26.64 Rs. 29.6, Image URL: https://d3313lwq5y3sh2.cloudfront.net/assets/photos/001/042/777/pharmacy_online_small/ASCARD_75MG_TAB__1.webp?1670306738
Name: Arinac Forte (400/60mg) 100 Tablets, Price: Rs. 114.93 Rs. 127.7, Image URL: https://d3313lwq5y3sh2.cloudfront.net/assets/photos/001/303/607/pharmacy_online_small/f.webp?1685971313
Name: Avil (25mg) 250 Tablets, Price: Rs. 14.29 Rs. 15.7, Image URL: https://d3313lwq5y3sh2.cloudfront.net/assets/photos/001/037/814/pharmacy_online_small/avil-25mg-tab-1005327__1.webp?1670232241
Name: ALCOHOL SWAB, Price: Rs. 5.4 Rs. 6.0, Image URL: https://d3313lwq5y3sh2.cloudfront.net/assets/photos/002/031/221/pharmacy_online_small/1476255.webp?1720009152
Name: Alp 

In [38]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# Set up Chrome in headless mode
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = "/usr/bin/google-chrome"  # Explicitly set the Chrome binary location

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Define JSON file path
json_file = '/content/medicines_data.json'

# Initialize an empty list to store medicine data
medicine_data = []

def scrape_medicines(letter):
    # Scrape the list of medicine details
    medicines = driver.find_elements(By.CSS_SELECTOR, 'a.show-loader')
    for medicine in medicines:
        try:
            # Attempt to extract details
            name = medicine.find_element(By.CSS_SELECTOR, 'span.name').text
            # Extract price and avoid `del` tag if present
            price_element = medicine.find_element(By.CSS_SELECTOR, 'span.price')
            price = price_element.text if not price_element.find_elements(By.TAG_NAME, 'del') else price_element.text.replace(price_element.find_element(By.TAG_NAME, 'del').text, '').strip()

            # Get image URL
            image_url = medicine.find_element(By.TAG_NAME, 'img').get_attribute('src')

            # Append the data to the medicine_data list
            medicine_data.append({
                'Letter': letter,
                'Name': name,
                'Price': price,
                'Image URL': image_url
            })
        except NoSuchElementException as e:
            print(f"Skipping medicine due to missing element: {e}")

def scrape_a_to_z():
    # Scrape data for each letter A to Z
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        url = f"https://healthwire.pk/pharmacy/medicines/{letter}"
        print(f"Scraping data for letter: {letter} ({url})")

        driver.get(url)
        sleep(2)  # Wait for the page to load

        # Scrape the medicine data with the current letter
        scrape_medicines(letter)

try:
    scrape_a_to_z()
finally:
    # Write all scraped data to the JSON file
    with open(json_file, mode='w', encoding='utf-8') as file:
        json.dump(medicine_data, file, indent=4, ensure_ascii=False)
    print(f"Data successfully saved to {json_file}")
    driver.quit()


Scraping data for letter: a (https://healthwire.pk/pharmacy/medicines/a)
Skipping medicine due to missing element: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.name"}
  (Session info: chrome=131.0.6778.264); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5926ac3c61fa <unknown>
#1 0x5926abed6810 <unknown>
#2 0x5926abf25506 <unknown>
#3 0x5926abf257a1 <unknown>
#4 0x5926abf19c46 <unknown>
#5 0x5926abf495ad <unknown>
#6 0x5926abf19b38 <unknown>
#7 0x5926abf4974e <unknown>
#8 0x5926abf68007 <unknown>
#9 0x5926abf49323 <unknown>
#10 0x5926abf17de0 <unknown>
#11 0x5926abf18dbe <unknown>
#12 0x5926ac39212b <unknown>
#13 0x5926ac3960c7 <unknown>
#14 0x5926ac37f6cc <unknown>
#15 0x5926ac396c47 <unknown>
#16 0x5926ac36467f <unknown>
#17 0x5926ac3b5288 <unknown>
#18 0x5926ac3b5450 <unknown>
#19 0x5926ac3c5076 <unknown>
#20 0x7f1ea3464a

In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

# Set up Chrome in headless mode
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = "/usr/bin/google-chrome"  # Explicitly set the Chrome binary location

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Define CSV file path
csv_file = '/content/medicines_data.csv'

# Write headers to CSV file
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Letter', 'Name', 'Price', 'Image URL', 'Page Number'])  # Added 'Page Number' column

def scrape_medicines(letter, page_number):
    # Scrape the list of medicine details
    medicines = driver.find_elements(By.CSS_SELECTOR, 'a.show-loader')
    for medicine in medicines:
        try:
            # Attempt to extract details
            name = medicine.find_element(By.CSS_SELECTOR, 'span.name').text
            # Extract price and avoid `del` tag if present
            price_element = medicine.find_element(By.CSS_SELECTOR, 'span.price')
            price = price_element.text if not price_element.find_element(By.TAG_NAME, 'del') else price_element.text.replace(price_element.find_element(By.TAG_NAME, 'del').text, '').strip()

            # Get image URL
            image_url = medicine.find_element(By.TAG_NAME, 'img').get_attribute('src')

            # Save the data with the current letter and page number
            with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow([letter, name, price, image_url, page_number])  # Write row with letter and page number
        except NoSuchElementException as e:
            print(f"Skipping medicine due to missing element: {e}")

def scrape_a_to_z():
    # Scrape data for each letter A to Z
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        page_number = 1  # Start at page 1 for each letter
        url = f"https://healthwire.pk/pharmacy/medicines/{letter}"
        print(f"Scraping data for letter: {letter} ({url})")

        driver.get(url)
        sleep(2)  # Wait for the page to load

        # Scrape the first page
        scrape_medicines(letter, page_number)

        # Load more and scrape additional pages until no more pages are found
        while True:
            try:
                # Locate and wait until the "Load More" button is clickable
                load_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, 'load-more-btn'))
                )

                # Scroll to the "Load More" button if necessary
                ActionChains(driver).move_to_element(load_more_button).perform()

                # Click the "Load More" button
                load_more_button.click()
                sleep(2)  # Wait for the new medicines to load
                page_number += 1
                print(f"Scraping page {page_number} for letter {letter}")

                # Scrape the medicines for the current page
                scrape_medicines(letter, page_number)
            except NoSuchElementException:
                print(f"No more pages for letter {letter} or error occurred.")
                break  # Stop if no more "Load More" button is found
            except ElementClickInterceptedException:
                print(f"Click intercepted for letter {letter}, trying again.")
                sleep(1)  # Wait for a while and retry clicking

try:
    scrape_a_to_z()
finally:
    driver.quit()


In [17]:
from google.colab import files
files.download('/content/medicines_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
from google.colab import files
files.download('/content/medicines_data.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
from git import Repo

repo_url = 'https://github.com/RaeesFatima/Web-scrapping.git'
local_dir = '/content/Web-scrapping'

# Clone the repository
!git clone {repo_url} {local_dir}

Cloning into '/content/Web-scrapping'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 167.35 KiB | 869.00 KiB/s, done.


In [24]:
import shutil

# Create 'Medicines' folder if not exists
!mkdir -p /content/Web-scrapping/Medicines

shutil.move('/content/medicines_data.csv', '/content/Web-scrapping/Medicines/medicines_data.csv')

'/content/Web-scrapping/Medicines/medicines_data.csv'

In [25]:
%cd /content/Web-scrapping

/content/Web-scrapping


In [26]:
!git add .

In [28]:
!git config --global user.email "raeesyousaf555@gmail.com"
!git config --global user.name "RaeesFatima"

In [29]:
!git commit -m "medicines data csv"

[main 31897a0] medicines data csv
 1 file changed, 474 insertions(+)
 create mode 100644 Medicines/medicines_data.csv


In [30]:
!git push https://github.com/RaeesFatima/Web-scrapping.git main

fatal: could not read Username for 'https://github.com': No such device or address
